# [IAPR][iapr]: Lab 2 ‒  Object description

**Group ID:** xx

**Author 1 (sciper):** Student Name 1 (xxxxx)  
**Author 2 (sciper):** Student Name 2 (xxxxx)   
**Author 3 (sciper):** Student Name 3 (xxxxx)   

**Release date:** 25.03.2022  
**Due date:** 08.04.2022 (11:59 pm)


## Important notes

The lab assignments are designed to teach practical implementation of the topics presented during class well as
preparation for the final project, which is a practical project which ties together the topics of the course.

As such, in the lab assignments/final project, unless otherwise specified, you may, if you choose, use external
functions from image processing/ML libraries like opencv and sklearn as long as there is sufficient explanation
in the lab report. For example, you do not need to implement your own edge detector, etc.

**! Before handling back the notebook <font color='red'> rerun </font>the notebook from scratch !**
`Kernel` > `Restart & Run All`

We will not rerun the notebook for you.


[iapr]: https://github.com/LTS5/iapr

---
## 0. Extract relevant data
We first need to extract the `lab-02-data.tar.gz` archive.
To this end, we use the [tarfile] module from the Python standard library.

[tarfile]: https://docs.python.org/3.6/library/tarfile.html

In [1]:
import cv2 as cv
import numpy as np
import time
import tarfile
import skimage
import os
from skimage import filters
from skimage import morphology
from skimage import measure
from skimage import exposure
from skimage import color
from skimage.transform import rescale
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.filters.rank import gradient
from skimage.morphology import disk
from skimage.exposure import histogram, cumulative_distribution, equalize_hist
from scipy import ndimage as ndi
from PIL import Image
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import webcolors
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tarfile
import os

data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-02-data'
data_part1 = os.path.join(data_base_path, data_folder, 'part1')
data_part2 = os.path.join(data_base_path, data_folder, 'part2')

tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
with tarfile.open(tar_path, mode='r:gz') as tar:
    tar.extractall(path=data_base_path)

---
## Part 1
In the `lab-02-data/part1` folder, you will find **28x28 grey-scale pictures** of handwritten "0" and "1".
These digits have been extracted from MNIST dataset (http://yann.lecun.com/exdb/mnist/).

Your goal is to extract, from each of those images, a 2-dimensional feature vector (i.e. 2 features) and to plot them all on a 2D graph.
If you have chosen good features, the vectors of the "0"'s should nicely cluster in one part of the plane and those of the "1"'s in another.

Please try:
1. Fourier Descriptors (15pts). 
    1. Implementation (10 pts).
    2. **Showing invariance to rotation, translation and scaling (5 pts).**
2. Additional method of your choice (5 pts)


**Note: for the Fourier descriptors, the u_k signal has to be constructed by following the contour point after point. Some pre-processing (image binarization, possibly some Mathematical Morphology) might be useful.**

### 1.1 Data visualization

In [ ]:
import skimage.io
import matplotlib.pyplot as plt
%matplotlib inline

def load(path, digit='0'):
    digit_path = os.path.join(path, digit)
    digit_names = [nm for nm in os.listdir(digit_path) if '.png' in nm]  # make sure to only load .png
    digit_names.sort()  # sort file names
    ic = skimage.io.imread_collection([os.path.join(digit_path, nm) for nm in digit_names])
    digit_im = skimage.io.concatenate_images(ic)
    return digit_im, digit_names
                        
#  Load zeros and ones
zeros_im, zeros_names = load(data_part1, digit='0')
ones_im, ones_names = load(data_part1, digit='1')
numberOf_im = zeros_im.shape[0]

# Plot images
fig, axes = plt.subplots(2, len(zeros_im), figsize=(12, 3))
for ax, im, nm in zip(axes[0], zeros_im, zeros_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
for ax, im, nm in zip(axes[1], ones_im, ones_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
    

In [ ]:
# function to plot multiple images
def plotMultipleImages(nrows, ncols, images, titles, cmap, figsize_x=14, figsize_y=7, suptitle=None):
    fig = plt.figure(figsize=(figsize_x,figsize_y))
    fig.suptitle(t=suptitle, y=0.63)
    fig.subplots_adjust(hspace=0.4, wspace=0.3)
    for i in range(len(titles)):
        ax = fig.add_subplot(nrows, ncols, i+1)
        if cmap[i]=='rgb':
            ax.imshow(images[i])
        else:
            ax.imshow(images[i], cmap=cmap[i])
        ax.set_title(titles[i])
    plt.show()

### 1.2 Fourier descriptors (15 pts)

#### 1.2.1 Preprocessing

Input image must be segmented and boundary of the object must be found out before calculating the Fourier Descriptors. Thus, appropriate image preprocessing methods are considered. 
- A low-pass filter should be used to filter out unnecessary informations that may lead to misclassification of numbers. Here, we select the `median filter` due to its main advantages: I) Median filtering preserves sharp contours which are quite important bases in Fourier descriptors, whereas linear low-pass filtering blurs such edges. II) Median filters are very efficient for smoothing of spiky noise.
- We check the histogram distribution of input image and found it is nicely bimodal. The two peaks (intensities of 0 and 255) correspond to the background and handwritten digits, and a few pixels lie in the range between 0 and 255. We then binariz the input images with thresholding to select areas of interest of images (here the digits). Because the input images are in different histrogram ditributions, a global threholding might not be good in all cases. Here, we use `Otsu's method` which determines an optimal global threshold value from the image histogram automatically.
- After that, we use `morphological opening` to remove some white patches caused by residual noise (can be found in 1_8.png).
- Finally, function `find_countours` is used to detect the contours of objects. And the returned value is a list of the coordinates of the contour pixels in order, going **counterclocwise** around the shape.

In [ ]:
# test image
ind = 7
test_im = ones_im[ind]
test_im_name = ones_names[ind]

# check the histrogram ditribution
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(test_im, cmap='gray')
ax[0].set_title(test_im_name)
ax[0].axis('off') 
ax[1].hist(test_im.ravel(), bins=256) 
ax[1].set_title('256 bins historgram of '+test_im_name)
ax[1].set_xlabel('Pixel intensity')
ax[1].set_ylabel('Number of pixels')
plt.show()

In [ ]:
def get_ordered_contour(in_im):
    # a list of the coordinates of the contour pixels (float type)
    
    contours = measure.find_contours(in_im, 5)
    ordered_contour = []
    contour_im = np.zeros(in_im.shape).astype(np.uint8)
    contour_len = []
    for contour in contours:
        contour_len.append(contour.shape[0])
        contour = np.rint(contour).astype(np.uint8)
        for x,y in contour:
            ordered_contour.append((x,y))
            contour_im[x,y] = 255
    #print(contour_len)
    return np.array(ordered_contour).astype(int), contour_im

def pre_processing(in_im, thresh=5):
    # De-noise
    # median filter with kernel = 1
    
    median_im = cv.medianBlur(in_im, 1)

    # Image binarization
    # Otsu's thresholding
    _,binary_im = cv.threshold(median_im, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)

    # Morphological opening
    morph_im = skimage.morphology.diameter_opening(binary_im,thresh)
    
    # get the orderred contours
    ordered_contour, contour_im = get_ordered_contour(morph_im)
    
    return median_im, binary_im, morph_im, contour_im, ordered_contour

def collect_contours_forAll(numberOf_im, zeros_im, ones_im, thresh=5):
    # compute the contour coordinates
    zeros_contour = []
    ones_contour = []
    zeros_contour_im = np.zeros(zeros_im.shape).astype(np.uint8)
    ones_contour_im = np.zeros(ones_im.shape).astype(np.uint8)
    for ind, zeros, ones in zip(range(numberOf_im), zeros_im, ones_im):
        _, _, _, contour_im, ordered_contour = pre_processing(zeros)
        zeros_contour_im[ind] = contour_im
        zeros_contour.append(ordered_contour)
        _, _, _, contour_im, ordered_contour = pre_processing(ones, thresh)
        ones_contour_im[ind] = contour_im
        ones_contour.append(ordered_contour)
    return zeros_contour, ones_contour, zeros_contour_im, ones_contour_im

In [ ]:
zeros_contour, ones_contour, zeros_contour_im, ones_contour_im = collect_contours_forAll(numberOf_im, zeros_im, ones_im)
# display the plots
plotMultipleImages(1, 10, zeros_im, zeros_names, cmap=['gray']*10, figsize_x=20, figsize_y=10,suptitle='Zeros images')
plotMultipleImages(1, 10, zeros_contour_im, zeros_names, cmap=['gray']*10, figsize_x=20, figsize_y=10, suptitle='Contours of Zeros images')
plotMultipleImages(1, 10, ones_im, ones_names, cmap=['gray']*10, figsize_x=20, figsize_y=10,suptitle='Ones images')
plotMultipleImages(1, 10, ones_contour_im, ones_names, cmap=['gray']*10, figsize_x=20, figsize_y=10, suptitle='Contours of Ones images')

#### 1.2.2 Implementation of Fourier descriptors

Fourier descriptors are a way of encoding the shape of a two-dimensional object by taking the Fourier transform of the boundary, where every  point on the boundary is mapped to a complex number. Let $(x_n, y_n)$ be the coordinates of the $n^{th}$ pixel on the contour of a given 2D shape, a complex number can be formed as: $C_n = x_n + j*y_n$. Now the Fourier Descriptors are calculated by combining complex array Fourier transform coefficients $C_0,C_1,C_2,......,C_{N-1}$. Thus, the following steps are followed:
- We firstly represent the `zeros_contour` and `ones_contour` obtained in previous part as an array of such complex numbers which corresponds to the pixels of the object if the image is placed in the complex plane.
- Then the pre-existing function `fft` in numpy is used to compute the Fourier descriptors $(F_0, F_1,...,F_{N-1})$. Users can select the first N descriptors which contain the majority of the shape information of the object as the results. 
- Because the first descriptor $F_0$ ("DC-component") of Fourier transform gives the average power of whole signal and is quite sensitive to the translation. Hence, we decided to truncate it to ensure location-invariance and select Fourier descriptors only from $F_1, F_2,...,F_{N-1}$.
- The fourier descriptors are complex here, thus we take their amplitudes as features.

In [ ]:
# index of selected fourier descriptors: start, start+1, ..., start+N-1
def get_Fourier_descriptors(real_contour, start=1, N=2, scale=None):
    complex_contour = np.zeros(real_contour.shape[0], dtype=complex)
    complex_contour.real = real_contour[:,0]
    complex_contour.imag = real_contour[:,1]
    
    descriptors = np.fft.fft(complex_contour)
    
    if scale:
        features = np.abs(descriptors[start:start+N])/np.abs(descriptors[1])
    else:
        features = np.abs(descriptors[start:start+N])
        
    return features

def get_features(zeros_contour, ones_contour, start=1, N=2, scale=None):
    
    zeros_features = []
    ones_features = []
    
    for zeros, ones in zip(zeros_contour, ones_contour):
        features = get_Fourier_descriptors(zeros, start, N, scale)
        zeros_features.append(features)
        features = get_Fourier_descriptors(ones, start, N, scale)
        ones_features.append(features)
        
    return np.array(zeros_features), np.array(ones_features)

def plot_features(zeros_features, ones_features, annote=None, title=None, plt_ax=None):
    if not plt_ax:
        plot_method= plt
        plt.xlabel('Amplitude of descriptor 1')
        plt.ylabel('Amplitude of descriptor 2')
    else:
        plot_method = plt_ax
        plot_method.set_xlabel('Amplitude of descriptor 1')
        plot_method.set_ylabel('Amplitude of descriptor 2')
    
    plot_method.scatter(zeros_features[:,0], zeros_features[:,1], label='digit 0')
    plot_method.scatter(ones_features[:,0], ones_features[:,1], label='digit 1')
    if annote:
        annote_text = [str(i) for i in range(10)]
        for text, zeros, ones in zip(annote_text, zeros_features, ones_features):
            plot_method.annotate(text, (zeros[0],zeros[1]), textcoords="offset points",
                         xytext=(0,10),
                         ha='center')
            plot_method.annotate(text, (ones[0],ones[1]), textcoords="offset points",
                         xytext=(0,10), 
                         ha='center')
    if title:
        if not plt_ax:
            plt.title(title)
        else:
            plot_method.set_title(title)
            
    plot_method.legend(loc='lower right')

In [ ]:
# take features
zeros_features, ones_features = get_features(zeros_contour, ones_contour)
plot_features(zeros_features, ones_features, annote=True, title='Feature space of digit clusters')

From the figure above, both digit clusters are well classified by using only two features of Fourier descriptors. It is possible to seperate these two clusters by using a simple linear classifier due to their high inter-variance. Additionally, the Fourier descriptors can capture the intra-variance for each cluster well. From raw images, we can see that most digit-1 objects (except for 1_5.png and 1_9.png) have similar contours, which means that the digit-1 cluster has a small intra-variance and thus it gives a compact form in feature space. For digit-0 cluster, due to the inconsistent written formats of digit-0 objects, it has a high intra-variance and thus it gives a sparse form in feature space, for example, slender 0s (i.e., 0_4.png and 0_9.png) give much lower energy in Fourier descriptors than those fat 0s (i.e., 0_5.png and 0_7.png), and 0_8.png becomes a outlier in feature space because it has much more contours than others.

#### 1.2.3 Properties of Fourier descriptor
In this section, rotation, translation and scaling are applied on raw images to check if Fourier descriptor is robust to image transformation.

##### 1.2.3.1 Rotation invariance
By Image rotation, the image is rotated about its center by specified number of degrees $(\theta)$, which can be defined by constructing a matrix in the form: $\begin{bmatrix} \cos\theta & -\sin\theta \\ \sin\theta & \cos\theta \end{bmatrix}$. Idealized rotation for the angle $(\theta)$ can be expressed as multiplication of the every element in the Fourer descriptors array with $e^{\theta{i}}$ . The discrete Fourier transform of such an array can be given as: $e^{\theta{i}}F_1, e^{\theta{i}}F_2,...,e^{\theta{i}}F_{N-1}$. Thus, for attaining the rotational invariance it is enough to take the absolute value of each descriptor, because $|e^{\theta{i}}| = 1$.


In [ ]:
def rotate_image(raw_im, degree = 45):
    '''
        To rotate image with certain degrees. 
        Degree is positive for anti-clockwise and negative for clockwise.
        
        Input: grayscale image, rotation degrees. 
        Output: rotated grayscale image. 
    '''
    # get the dimensions of the image
    (height, width) = raw_im.shape
    # calculate the center of the image
    center = (width // 2, height // 2)
    # get a rotation mask
    mask = cv.getRotationMatrix2D(center, degree, 1.0)
    # rotate the image
    rotated_im = cv.warpAffine(src=raw_im, M=mask, dsize=(width,height))
    
    return rotated_im

Here we test 

In [ ]:
# test the rotate_image function on images
zeros_rotate_im = np.zeros(zeros_im.shape).astype(np.uint8)
ones_rotate_im = np.zeros(ones_im.shape).astype(np.uint8)
degrees = [20, 40, 60]
fig, axes = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(15,8))
ax = axes.ravel()
zeros_features, ones_features = get_features(zeros_contour, ones_contour)
plot_features(zeros_features, ones_features, annote=True, title='Original feature space of digit clusters', plt_ax=ax[0])
for i, degree in enumerate(degrees):
    for ind, zeros, ones in zip(range(numberOf_im), zeros_im, ones_im):
        zeros_rotate_im[ind] = rotate_image(zeros, degree)
        ones_rotate_im[ind] = rotate_image(ones, degree)
    zeros_rotate_contour, ones_rotate_contour, _ , _ = collect_contours_forAll(numberOf_im, zeros_rotate_im, ones_rotate_im)
    zeros_rotate_features, ones_rotate_features = get_features(zeros_rotate_contour, ones_rotate_contour)
    plot_features(zeros_rotate_features, ones_rotate_features, annote=True, title='Feature space of digit clusters with {0} degrees rotation'.format(degree), plt_ax=ax[i+1])

Since we use the absolute values of the Fourier descriptors, rotation invariance is already present. Rotation of the contour is simply a phase shift, which does not factor into the absolute value. In order to demonstrate this, the images are rotated by random degrees and their fourier descriptors are displayed again.

##### 1.2.3.2 Translation invariance
Translation refers to the rectilinear shift of an object i.s. an image from one location to another. If we know the amount of shift in horizontal and the vertical direction, say $(t_x, t_y)$ then we make a transformation matrix e.g. $\begin{bmatrix} 1 & 0 & t_x \\ 0 & 1 & t_y \end{bmatrix}$. According to the theory [1], the translation affects only the value of discrete Fourier transform’s first element, and the translation invariance can be acquired just by truncating the first element $F_0$. Thus, we use 

In [ ]:
def translate_image(raw_im, tx, ty):
    '''
        To translate image with distances tx, ty. 
        
        Input: grayscale image, translation distance. 
        Output: translated grayscale image. 
    '''
    # get the dimensions of the image
    (height, width) = raw_im.shape
    # get a translation mask
    mask = np.float32([[1, 0, tx], [0, 1, ty]])
    # translate the image
    translate_im = cv.warpAffine(src=raw_im, M=mask, dsize=(width,height))
    
    return translate_im.astype(np.uint8)

In [ ]:
# test image
ind = 8
test_im = zeros_im[ind]
translate_dist = [2, 3]
test_translate_im = translate_image(test_im, translate_dist[0], translate_dist[1])
images = [test_im, test_translate_im]
title = ['ori', 'tra']
plotMultipleImages(1, 2, images, title, cmap=['gray']*2, figsize_x=10, figsize_y=7)

Here we use conservative translation distances because a significant translation might cause parts of image objects to be cropped/hidden (i.e. out of scene) which will definitely change the Fourier descriptors.

In [ ]:
# test the translate_image function on images
zeros_translate_im = np.zeros(zeros_im.shape).astype(np.uint8)
ones_translate_im = np.zeros(ones_im.shape).astype(np.uint8)
translate_dists = [[2, 3], [-2, -3], [-4, 4]]
fig, axes = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(15,8))
ax = axes.ravel()
zeros_features, ones_features = get_features(zeros_contour, ones_contour)
plot_features(zeros_features, ones_features, annote=True, title='Original feature space of digit clusters', plt_ax=ax[0])
for i, translate_dist in enumerate(translate_dists):
    for ind, zeros, ones in zip(range(numberOf_im), zeros_im, ones_im):
        zeros_translate_im[ind] = translate_image(zeros, translate_dist[0], translate_dist[1])
        ones_translate_im[ind] = translate_image(ones, translate_dist[0], translate_dist[1])
    zeros_translate_contour, ones_translate_contour, _ , _ = collect_contours_forAll(numberOf_im, zeros_translate_im, ones_translate_im)
    zeros_translate_features, ones_translate_features = get_features(zeros_translate_contour, ones_translate_contour)
    plot_features(zeros_translate_features, ones_translate_features, annote=True, title='Feature space of digit clusters with {0} translation distance'.format(translate_dist), plt_ax=ax[i+1])

##### 1.2.3.3 Scaling invariance
Rescaling an image means changing the dimensions of it, be it width alone, height alone or changing both of them. Here we use `rescale` function from skimage library to resize an image by a given scaling factor. Idealized scaling can be expressed as multiplication of the every element in the array with the real constant $C$

In [ ]:
# test image
ind = 9
test_im = zeros_im[ind]
scale_factors = [0.25, 0.5, 2, 4]
test_scale_im_0 = (rescale(test_im, scale_factors[0], preserve_range=True, anti_aliasing=True))
test_scale_im_1 = (rescale(test_im, scale_factors[1], preserve_range=True, anti_aliasing=True))
test_scale_im_2 = (rescale(test_im, scale_factors[2], preserve_range=True)).astype(np.uint8)
test_scale_im_3 = (rescale(test_im, scale_factors[3], preserve_range=True)).astype(np.uint8)
images = [test_im, test_scale_im_0, test_scale_im_1, test_scale_im_2, test_scale_im_3]
title = ['ori', 'tra_0', 'tra_1', 'tra_2', 'tra_3']
plotMultipleImages(1, 5, images, title, cmap=['gray']*5, figsize_x=10, figsize_y=7)
print(np.round(np.dot(test_im.shape,0.2)), test_scale_im_0.shape, test_scale_im_1.shape, test_scale_im_2.shape, test_scale_im_3.shape)


In [ ]:
scale_factors = [1, 0.5, 2, 4]
threshs = [5, 5, 10, 20]
fig, axes = plt.subplots(2, 2, sharex=False, sharey=False, figsize=(15,10))
ax = axes.ravel()
#zeros_features, ones_features = get_features(zeros_contour, ones_contour, start=2, N=2, scale=True)
#plot_features(zeros_features, ones_features, annote=True, title='Original feature space of digit clusters', plt_ax=ax[0])
for i, scale_factor in enumerate(scale_factors):
    print('factor is ',scale_factor)
    scale_shape = (10, int(28*scale_factor), int(28*scale_factor))
    zeros_scale_im = np.zeros(scale_shape).astype(np.uint8)
    ones_scale_im = np.zeros(scale_shape).astype(np.uint8)
    for ind, zeros, ones in zip(range(numberOf_im), zeros_im, ones_im):
        zeros_scale_im[ind] = rescale(zeros, scale_factor, preserve_range=True, anti_aliasing=False)
        ones_scale_im[ind] = rescale(ones, scale_factor, preserve_range=True, anti_aliasing=False)
    zeros_scale_contour, ones_scale_contour, _ , _ = collect_contours_forAll(numberOf_im, zeros_scale_im, ones_scale_im, thresh=threshs[i])
    zeros_scale_features, ones_scale_features = get_features(zeros_scale_contour, ones_scale_contour, start=2, N=2, scale=True)
    plot_features(zeros_scale_features, ones_scale_features, annote=True, title='Feature space of digit clusters with scale factor = {0}'.format(scale_factor), plt_ax=ax[i])

### 1.3 Additional method (5 pts)

In [ ]:
# Add your implementation and discussion

---
## Part 2
The `lab-02-data/part2` folder contains grey-scale pictures of handwritten "2" and "3".
Extract the same feature (typically 2 Fourier descriptors) as in part 1 also on these images and plot them on the same graph as the features of the "0" and "1".
Is it possible to discriminate all these 4 digits with a 2-dimensional feature vector?

### 2.1 Data visualization

In [ ]:
#  Load twos and threes
twos_im, twos_names = load(data_part2, digit='2')
threes_im, threes_names = load(data_part2, digit='3')

# Plot images
fig, axes = plt.subplots(2, len(twos_im), figsize=(12, 3))
for ax, im, nm in zip(axes[0], twos_im, twos_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)
for ax, im, nm in zip(axes[1], threes_im, threes_names):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(nm)

### 2.2 Fourier descriptors - 4 digits (10 pts)

In [ ]:
# Add your implementation and discussion